## Google Colab Rust Setup

The following cell is used to set up a Rust environment on Colab. Don't execute it locally!

Many thanks to [`mateusvmv`](https://github.com/mateusvmv) for this hack in [`gist.github.com/korakot/ae95315ea6a3a3b33ee26203998a59a3`](https://gist.github.com/korakot/ae95315ea6a3a3b33ee26203998a59a3?permalink_comment_id=4715636#gistcomment-4715636).

In [ ]:
# This script sets up and spins up a Jupyter Notebook environment with a Rust kernel using Nix and IPC Proxy. 
!wget -qO- https://gist.github.com/wiseaidev/2af6bef753d48565d11bcd478728c979/archive/3f6df40db09f3517ade41997b541b81f0976c12e.tar.gz | tar xvz --strip-components=1
!bash setup_evcxr_kernel.sh

## Install Dependencies

In [2]:
:dep chrono = { version = "0.4.31"}
// or
// :dep chrono = { git = "https://github.com/chronotope/chrono" }

In [3]:
:dep polars = { version = "0.35.0", features = ["describe", "lazy", "ndarray", "object", "dtype-struct", "concat_str", "mode"] }
// or
// :dep blas-src = { git = "https://github.com/pola-rs/polars", features = ["describe", "lazy", "ndarray", "object", "dtype-struct", "concat_str", "mode"]}

<hr />

## Import Libraries

In [4]:
use polars::datatypes::DataType;
use polars::prelude::*;
use chrono::{DateTime, NaiveDateTime, TimeZone, Utc};
use std::sync::Arc;

<hr />

## Series Object

In [5]:
let series1: Series = [1, 2, 3].iter().collect();
// or
// let series1: Series = Series::new("", &[1, 2, 3]);
series1

shape: (3,)
Series: '' [i32]
[
	1
	2
	3
]

In [6]:
let seasons_ser: Series = Series::new("seasons", &["Winter", "Spring", "Summer", "Fall"]);
seasons_ser

shape: (4,)
Series: 'seasons' [str]
[
	"Winter"
	"Spring"
	"Summer"
	"Fall"
]

In [7]:
let s: Series = Series::new("seasons", &[None, Some(1), Some(2)]);
s

shape: (3,)
Series: 'seasons' [i32]
[
	null
	1
	2
]

In [8]:
Some(f64::NAN)==None

false

In [9]:
f64::NAN==f64::NAN

false

In [10]:
let s: Series = Series::new("seasons", &[None, Some(1), Some(2)]);
s.null_count()

1

In [11]:
s.drop_nulls()

shape: (2,)
Series: 'seasons' [i32]
[
	1
	2
]

In [12]:
let s: Series = Series::new("numbers", &[Some(f64::NAN), Some(1.), Some(2.)]);
s.cast(&DataType::Int64).unwrap()

shape: (3,)
Series: 'numbers' [i64]
[
	null
	1
	2
]

## Series Creation

In [13]:
let s = Series::new_empty("Height", &DataType::Float32);
s

shape: (0,)
Series: 'Height' [f32]
[
]

In [14]:
let s: Series = Series::new("employees", &["Mahmoud", "Ferris"]);
s

shape: (2,)
Series: 'employees' [str]
[
	"Mahmoud"
	"Ferris"
]

In [15]:
let s: Series = Series::new("employees", &["Mahmoud", "Ferris"]);
s.name()

"employees"

In [16]:
let s: Series = Series::new("employees", &vec!["Mahmoud", "Ferris"]);
s

shape: (2,)
Series: 'employees' [str]
[
	"Mahmoud"
	"Ferris"
]

In [17]:
let s = Float64Chunked::new("b", &[1., 2., 3.]).into_series();
s

shape: (3,)
Series: 'b' [f64]
[
	1.0
	2.0
	3.0
]

## Datetime Type

In [18]:
let date: DateTime<Utc> = Utc.with_ymd_and_hms(2020, 1, 1, 0, 0, 0).unwrap();
let s = Series::new("b", &[date.date_naive()]);
s

shape: (1,)
Series: 'b' [date]
[
	2020-01-01
]

## Indexing & Slicing

In [19]:
let s: Series = Series::new("employees", &vec!["Ferris", "Mahmoud"]);
s

shape: (2,)
Series: 'employees' [str]
[
	"Ferris"
	"Mahmoud"
]

In [20]:
let s: Series = Series::new("employees", &vec!["Mahmoud", "Ferris"]);
s.slice(0, 1)

shape: (1,)
Series: 'employees' [str]
[
	"Mahmoud"
]

## Length

In [21]:
let s: Series = Series::new("employees", &vec!["Mahmoud", "Ferris"]);
s.len()

2

## Reverse

In [22]:
let s: Series = Series::new("employees", &vec!["Mahmoud", "Ferris"]);
s.reverse()

shape: (2,)
Series: 'employees' [str]
[
	"Ferris"
	"Mahmoud"
]

## Empty

In [23]:
let s: Series = Series::new("employees", &vec!["Mahmoud", "Ferris"]);
s.is_empty()

false

## Drop nulls

In [24]:
let s: Series = Series::new("employees", &vec![Some("Ferris"), None]);
s.drop_nulls()

shape: (1,)
Series: 'employees' [str]
[
	"Ferris"
]

## Summarizing Series

In [25]:
let s = Series::new("Measurements", &[-1.01,  0.86, -4.60, 3.98,  0.53, -7.04, 3.98,  0.53, -7.04, 0.86, 0.16, 0.26, 0.81]);

// First 10 rows
s.head(None)

shape: (10,)
Series: 'Measurements' [f64]
[
	-1.01
	0.86
	-4.6
	3.98
	0.53
	-7.04
	3.98
	0.53
	-7.04
	0.86
]

In [26]:
let s = Series::new("Measurements", &[-1.01,  0.86, -4.60, 3.98,  0.53, -7.04, 3.98,  0.53, -7.04, 0.86, 0.16, 0.26, 0.81]);

// First 5 rows
s.head(Some(5))

shape: (5,)
Series: 'Measurements' [f64]
[
	-1.01
	0.86
	-4.6
	3.98
	0.53
]

In [27]:
let s = Series::new("Measurements", &[-1.01,  0.86, -4.60, 3.98,  0.53, -7.04, 3.98,  0.53, -7.04, 0.86, 0.16, 0.26, 0.81]);

// Last 10 rows
s.tail(None)

shape: (10,)
Series: 'Measurements' [f64]
[
	3.98
	0.53
	-7.04
	3.98
	0.53
	-7.04
	0.86
	0.16
	0.26
	0.81
]

## Append

In [28]:
let mut s1 = Series::new("Age", &[23., 27.]);
let s2 = Series::new("Height", &[1.84, 1.78]);
s1.append(&s2)

Ok(shape: (4,)
Series: 'Age' [f64]
[
	23.0
	27.0
	1.84
	1.78
])

## Cast

In [29]:
let mut s = Series::new("Measurements", &[-1.01,  0.86, -4.60, 3.98,  0.53, -7.04, 3.98,  0.53, -7.04]);
s.cast(&DataType::Int32).unwrap()

shape: (9,)
Series: 'Measurements' [i32]
[
	-1
	0
	-4
	3
	0
	-7
	3
	0
	-7
]

## Fill Nulls

### 1. Forward fill

In [30]:
let s = Series::new("some_missing", &[Some(1), None, Some(3), Some(4), None, Some(6)]);
let filled = s.fill_null(FillNullStrategy::Forward(None)).unwrap();
filled

shape: (6,)
Series: 'some_missing' [i32]
[
	1
	1
	3
	4
	4
	6
]

### 2. Backward fill

In [31]:
let s = Series::new("some_missing", &[Some(1), None, Some(3), Some(4), None, Some(6)]);
let filled = s.fill_null(FillNullStrategy::Backward(None)).unwrap();
filled

shape: (6,)
Series: 'some_missing' [i32]
[
	1
	3
	3
	4
	6
	6
]

### 3. Mean fill

In [32]:
let s = Series::new("some_missing", &[Some(1), None, Some(3), Some(4), None, Some(6)]);
let filled = s.fill_null(FillNullStrategy::Mean).unwrap();
filled

shape: (6,)
Series: 'some_missing' [i32]
[
	1
	3
	3
	4
	3
	6
]

### 4. Min fill

In [33]:
let s = Series::new("some_missing", &[Some(1), None, Some(3), Some(4), None, Some(6)]);
let filled = s.fill_null(FillNullStrategy::Min).unwrap();
filled

shape: (6,)
Series: 'some_missing' [i32]
[
	1
	1
	3
	4
	1
	6
]

### 5. Max fill

In [34]:
let s = Series::new("some_missing", &[Some(1), None, Some(3), Some(4), None, Some(6)]);
let filled = s.fill_null(FillNullStrategy::Max).unwrap();
filled

shape: (6,)
Series: 'some_missing' [i32]
[
	1
	6
	3
	4
	6
	6
]

## Sampling

In [35]:
let s = Series::new("Measurements", &[-1.01,  0.86, -4.60, 3.98,  0.53, -7.04, 3.98,  0.53, -7.04]);
// Take 4 samples with replacement and with shuffle (we are talkin' probability here).
s.sample_n(4, true, true, Some(9999))

Ok(shape: (4,)
Series: 'Measurements' [f64]
[
	0.53
	-1.01
	3.98
	0.53
])

## Descriptive statistics

### Measures of central tendency

### 1. Mean

In [36]:
let s = Series::new("Measurements", &[-1.01,  0.86, -4.60, 3.98,  0.53, -7.04, 3.98,  0.53, -7.04]);
s.mean().unwrap()

-1.09

### 2. Median

In [37]:
let s = Series::new("Measurements", &[-1.01,  0.86, -4.60, 3.98,  0.53, -7.04, 3.98,  0.53, -7.04]);
s.median().unwrap()

0.53

### 3. Mode

In [40]:
let s = Series::new("Measurements", &[-1.01,  0.86, -4.60, 3.98,  0.53, -7.04, 3.98,  0.53, -7.04]);
mode::mode(&s).unwrap()

shape: (3,)
Series: 'Measurements' [f64]
[
	-7.04
	3.98
	0.53
]

### Measures of spread

### 1. Interquartile range

In [41]:
let s = Series::new("Measurements", &[-1.01,  0.86, -4.60, 3.98,  0.53, -7.04, 3.98,  0.53, -7.04]);
s.quantile_as_series(0.75, QuantileInterpolOptions::Nearest).unwrap()

shape: (1,)
Series: 'Measurements' [f64]
[
	0.86
]

---
---